In [1]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
from sqlalchemy import create_engine
import dfutils

In [2]:
#DB_NAME = 'wic'
#WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
#DB_USER = 'aduprey'
#DB_PASSWORD = 'aduprey'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'root'
DB_PASSWORD = 'waterislandcapital'
DB_NAME = 'wic'
engine = create_engine("mysql://" + DB_USER + ":" + DB_PASSWORD + "@" + WIC_DB_HOST + "/wic")

In [3]:
def optimized_execute(query, commit=False):
    '''
    Executes the search query in Northpoint
    '''
    
    with closing(pymssql.connect(host='10.16.1.16', user='readonly_user', password='readonly_user',
                                 database='PnLAppDb')) as np_pnl_conn:
        with closing(np_pnl_conn.cursor()) as np_pnl_cursor:
            np_pnl_cursor.execute(query)
            if commit:
                np_pnl_conn.commit()
                return

            fetched_res = np_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
            return fetched_res

In [4]:
def tradar_optimized_execute(query, commit=False):
    try:
        with closing(pymssql.connect(host='NYDC-WTRTRD01', user='paz', password='Welcome2',
                                     database='TradarBE')) as trdr_pnl_conn:
            with closing(trdr_pnl_conn.cursor()) as trdr_pnl_cursor:
                trdr_pnl_cursor.execute(query)
                if commit:
                    trdr_pnl_conn.commit()
                    return

                fetched_res = trdr_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
                return fetched_res
    except Exception as e:
        print e

In [5]:
def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res


In [6]:
def get_active_tradegroups_from_tradar():
    query = "SELECT DISTINCT " \
            "MAX(A.timeKey) as MaxDate, " \
            "F.fund AS Fund, " \
            "B.strat AS TradeGroup, " \
            "S.ticker, " \
            "ST.sectype, " \
            "ST.secTypeDesc " \
            "FROM performanceAttributionFact AS A " \
            "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
            "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
            "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
            "INNER JOIN dbo.SecType ST on S.sectype = ST.sectype " \
            "WHERE A.timeKey = (SELECT MAX(timekey) FROM dbo.PerformanceAttributionFact WHERE timekey < (SELECT max(timekey) FROM dbo.PerformanceAttributionFact)) " \
            "AND ST.secTypeDesc NOT IN ('Exchrate') " \
            "AND F.fund IN ('ARB', 'AED', 'TACO', 'WED', 'CAM', 'LG', 'TAQ', 'LEV', 'MACO', 'MALT') " \
            "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.sectype, ST.secTypeDesc " \
    
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'SecTypeDesc']
    query_result = tradar_optimized_execute(query)
    df = pd.DataFrame(query_result, columns=cols)
    unique_combos_df = df.drop_duplicates(['Fund', 'TradeGroup'])
    return unique_combos_df.reset_index(drop=True)

In [7]:
def get_max_date(table_name):
    query = "SELECT MAX(df.Date) FROM wic." + table_name + " AS df "
    results = wic_optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [8]:
def get_position_calculated_values_history_max_date():  # done
    query = "SELECT MAX(PCVH.TradeDate) FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH "
    results = optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [9]:
def get_position_calculated_values_history(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None,
                                           limit_to_tradegroups=[], is_tg_names_in_tradar_format=False,
                                           fund_code=None, NP_qty_rollup=True):
    if start_date_yyyy_mm_dd is None or end_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '2019-03-01'
        end_date_yyyy_mm_dd = get_position_calculated_values_history_max_date()
    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND PCVH.TradeGroup IN (" + ",".join([("'" + tg + "'") 
                                                                   for tg in limit_to_tradegroups 
                                                                   if tg is not None]) + ") "
        if is_tg_names_in_tradar_format: 
            tg_filter_clause = tg_filter_clause.replace('PCVH.TradeGroup', 'SUBSTRING(PCVH.TradeGroup,0,21)')
            
    fund_code_filtuer_clause = " " if fund_code is None else " AND PCVH.FundCode = '" + fund_code + "' "
    # region query
    # New Fund by added bv Kshitij "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL":"MACO"
    query = "SELECT " \
            "PCVH.TradeDate, " \
            "ISNULL(PCVH.FundCode, " \
            "CASE F.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'WED' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END " \
            ") AS FundCode, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_Tradar_Name, " \
            "PCVH.[Marketing GROUP] AS Sleeve,   " \
            "ISNULL(PCVH.[Bucket],'NA') AS Bucket, " \
            "PCVH.TradeGroupId, " \
            "SPT.Ticker, " \
            "SPT.SecType, " \
            "ISNULL(SPT.MarketCapCategory, 'N/A') AS MarketCapCategory, " \
            "PCVH.SecurityID, " \
            "PCVH.DealTermsCash, " \
            "PCVH.DealTermsStock, " \
            "PCVH.DealValue, " \
            "PCVH.DealClosingDate, " \
            "CASE " \
            "WHEN SPT.SecType = 'FxFwd' THEN 0.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Equity Special Situations'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Opportunistic'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Merger Arbitrage'  THEN " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN " \
            "CASE PCVH.DealValue WHEN 0 THEN 0.0 ELSE PCVH.DealTermsStock/PCVH.DealValue END " \
            "ELSE 1.0 END " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Credit Opportunities'  THEN " \
            "CASE WHEN SPT.SecType IN ('EQ','ExchOpt') THEN 1.0 ELSE 0.0 END " \
            "ELSE NULL END " \
            "AS [Equity Risk Factor], " \
            "PCVH.DV_01, " \
            "PCVH.CR_01, " \
            "PCVH.Adj_CR_01, " \
            "ISNULL(SPT.UltimateCountry,'N/A') AS UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA') AS AlphaHedge, " \
            "SUM(PCVH.ExposureLong_USD) AS Exposure_Long, " \
            "SUM(PCVH.ExposureShort_USD) AS Exposure_Short, " \
            "SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD) AS Exposure_Net, " \
            "CASE WHEN NAV.NAV = 0 THEN NULL " \
            "ELSE 100.0*(SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD)/NAV.NAV) END " \
            "AS [Exposure_Net(%)], " \
            "SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD) AS NetMktVal, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD)) ELSE NULL END AS Capital, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN 100.0*(ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD))/NAV.NAV) ELSE NULL " \
            "END AS [Capital % of NAV], " \
            "100.0*SUM(PCVH.Base_Case_NAV_Impact) AS BaseCaseNavImpact, " \
            "100.0*SUM(PCVH.Outlier_NAV_Impact) AS OutlierNavImpact, " \
            "SUM(PCVH.Qty) AS QTY, " \
            "ISNULL(PCVH.TradeGroupLongShortFlag,'NA') AS LongShort, " \
            "ISNULL(SPT.GICSSectorName,'NA') AS Sector, " \
            "ISNULL(SPT.GICSIndustryName,'NA') AS Industry, " \
            "ISNULL(PCVH.TradeGroupCatalystRating, 'NA') AS CatalystRating, " \
            "CASE ISNULL(PCVH.TradeGroupCatalystRating, 'NA') " \
            "   WHEN 'NA' THEN 'N/A' " \
            "   WHEN 0 THEN 'No Catalyst' " \
            "   WHEN 1 THEN 'Soft' " \
            "   WHEN 2 THEN 'Binary' " \
            "   WHEN 3 THEN 'Hard' " \
            "   ELSE 'Unknown' END AS CatalystName, " \
            "NAV.NAV " \
            "FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH " \
            "INNER JOIN PnLAppDb.dbo.Funds AS F ON PCVH.Portfolio = F.FundID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON PCVH.SecurityID = SPT.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "LEFT OUTER JOIN PnLAppDb.pnl.DailyNAV AS NAV ON PCVH.Portfolio = NAV.FundId " \
            "AND PCVH.TradeDate = NAV.[DATE] " \
            "WHERE  TradeDate >= '" + start_date_yyyy_mm_dd + "' AND TradeDate <= '" + end_date_yyyy_mm_dd + "' " \
            + tg_filter_clause + \
            fund_code_filtuer_clause + \
            "GROUP BY " \
            "F.FundName, PCVH.FundCode,PCVH.TradeDate, PCVH.[Marketing GROUP], SPT_TRDGRP.SecName, " \
            "PCVH.TradeGroupId, SPT.Ticker, SPT.BloombergGlobalId, SPT.BloombergGID, " \
            "SPT.SecType, ISNULL(SPT.MarketCapCategory, 'N/A'), " \
            "PCVH.SecurityId, SPT.UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA'),PCVH.TradeGroupLongShortFlag, ISNULL(SPT.GICSSectorName,'NA'), " \
            "ISNULL(SPT.GICSIndustryName,'NA'), ISNULL(PCVH.TradeGroupCatalystRating, 'NA'), " \
            "ISNULL(PCVH.[Bucket],'NA'), PCVH.DealTermsCash, PCVH.DealTermsStock, " \
            "PCVH.DealValue, PCVH.DealClosingDate, " \
            "PCVH.DV_01, PCVH.CR_01, PCVH.Adj_CR_01, NAV.NAV, PCVH.Analyst " \
            "ORDER BY PCVH.TradeDate "
    # endregion
    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "FundCode", "TradeGroup", "TradeGroup_Tradar_Name", "Sleeve",
                                        "Bucket", "TradeGroupId", "Ticker", "SecType", "MarketCapCategory",
                                        "SecurityId", "DealTermsCash", "DealTermsStock", "DealValue", 
                                        "DealClosingDate", "Equity_Risk_Factor", "DV01", "CR01", "Adj_CR01",
                                        "UltimateCountry", "AlphaHedge", "Exposure_Long", "Exposure_Short",
                                        "Exposure_Net", "Exposure_Net(%)", "NetMktVal", "Capital($)", 
                                        "Capital(%)", "BaseCaseNavImpact", "OutlierNavImpact", "Qty", 
                                        "LongShort", "Sector", "Industry", "CatalystRating", "CatalystName", "Fund_NAV"])

    float_cols = ['NetMktVal', 'Capital($)', 'Capital(%)', 'Exposure_Net', 'Exposure_Net(%)', 'BaseCaseNavImpact']
    df[float_cols] = df[float_cols].astype(float)
    df['Equity_Risk_Exp'] = df['Equity_Risk_Factor'].astype(float)*df['Exposure_Net'].astype(float)
    df[['DV01', 'CR01', 'Adj_CR01']] = df[['DV01', 'CR01', 'Adj_CR01']].fillna(0).astype(float)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))
    df["DealClosingDate"] = df["DealClosingDate"].apply(lambda x: pd.to_datetime(x))

    as_of_dt = datetime.datetime.strptime(end_date_yyyy_mm_dd, '%Y-%m-%d')

    def duration_calc(days):
        if days <= 90:
            return '0M-3M'
        if days <= 180:
            return '3M-6M'
        if days <= 365:
            return '6M-12M'
        return 'Yr+'

    df["DealDuration"] = df["DealClosingDate"].apply(lambda x: None if pd.isnull(x) else
                                                     duration_calc((as_of_dt - pd.to_datetime(x)).days))

    # region ADJUSTING FOR POSTED TRADES AFTER TRADE DATE - AFFECTING QTY ONLY
    # adjust for end_date_yyyy_mm_dd - take trades with tradedate = end_date_yyyy_mm_dd and posted_date
    #AFTER end_date_yyyy_mm_dd
    # adjust the Qty into the pcvh_df
    if NP_qty_rollup:
        qty_adjust_query = "SELECT " \
                            "A.TradeDate AS [DATE], " \
                            "CASE A.Fund " \
                            "WHEN 'Columbia' THEN 'CAM' " \
                            "WHEN 'Litman Gregory' THEN 'LG' " \
                            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                            "WHEN 'TransAmerica' THEN 'TAF' " \
                            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
                            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL COMMINGLED MASTER FUND LP' THEN 'MACO' " \
                            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
                            "END AS FundCode, " \
                            "A.TradeGroupId, " \
                            "A.SecurityId, " \
                            "SUM((CASE SPT.SecType WHEN 'ExchOpt' THEN 100.0 ELSE 1.0 END)*A.Shares) AS Qty " \
                            "FROM " \
                            "[PnLAppDb].[dbo].[vTradesFlatView] AS A " \
                            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON A.SecurityId = SPT.ID " \
                            "WHERE PostedDate >= DATEADD(DAY,1,TradeDate) AND TradeDate >= '" \
                            + start_date_yyyy_mm_dd + "' " \
                            "GROUP BY A.TradeDate, A.SecurityId, A.Ticker, A.Fund, A.TradeGroupId, SPT.SecType "
        results = optimized_execute(qty_adjust_query)
        qty_adjust_df = pd.DataFrame(results, columns=['Date', 'FundCode', 'TradeGroupId', 'SecurityId', 'Qty_adj'])
        qty_adjust_df[['TradeGroupId', 'SecurityId', 'Qty_adj']] = qty_adjust_df[['TradeGroupId',
                                                                                  'SecurityId', 'Qty_adj']].astype(float)
        qty_adjust_df['Date'] = qty_adjust_df['Date'].apply(lambda x: pd.to_datetime(x))

        df[['TradeGroupId', 'SecurityId', 'Qty']] = df[['TradeGroupId', 'SecurityId', 'Qty']].astype(float)
        df = pd.merge(df, qty_adjust_df, how='left', on=['Date', 'FundCode', 'TradeGroupId', 'SecurityId'])
        df['Qty'] = df['Qty'] + df['Qty_adj'].fillna(0)
        del df['Qty_adj']
    # endregion
    print('PCVH retrieval completed....')
    return df

In [42]:
def get_NAV_df_by_date(fund_name = None, start_date_yyyy_mm_dd=None):
    if start_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '20150101'
    elif (start_date_yyyy_mm_dd is not None) & (type(start_date_yyyy_mm_dd) != str):
        start_date_yyyy_mm_dd = start_date_yyyy_mm_dd.strftime('%Y%m%d')

    query = "SELECT A.[DATE], A.NAV, " \
            "CASE B.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END AS FundCode " \
            "FROM PnLAppDb.pnl.DailyNAV AS A " \
            "INNER JOIN PnLAppDb.dbo.Funds AS B ON A.FundId = B.FundID "
 
    if fund_name is not None and start_date_yyyy_mm_dd is not None:
        query += "WHERE  A.[Date] >= '" + start_date_yyyy_mm_dd + "' AND B.FundName = '" + fund_name + "' "
    elif fund_name is not None:
        query += " WHERE B.FundName = '" + fund_name + "' "
    elif start_date_yyyy_mm_dd is not None:
        query += "WHERE  A.[Date] >= '" + start_date_yyyy_mm_dd + "' "

    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "NAV", "FundCode"])
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df.index.name = 'Date'
    df = df.sort_index()
    df['NAV'] = df['NAV'].astype(float)
    #idx = pd.date_range(df['Date'].min(), df['Date'].max())
    #df = df.reindex(idx, fill_value=np.nan)
    del df['Date']
    df.reset_index(level=0, inplace=True)
    df.rename(columns={"index": "Date"}, inplace=True)
    df.index.name = 'Date'
    df.ffill(inplace=True)
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    # df.reset_index(inplace=True)
    df = df.reset_index(level=0, drop=True).reset_index()  # Added by Kshitij
    del df['index']
    return df[df['Date'] <= (datetime.datetime.today() - datetime.timedelta(days=1))]


In [56]:
def get_spread_history_by_date(start_date_yyyy_mm_dd=None):
    if start_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '20150101'
    elif (start_date_yyyy_mm_dd is not None) & (type(start_date_yyyy_mm_dd) != str):
        start_date_yyyy_mm_dd = start_date_yyyy_mm_dd.strftime('%Y%m%d')
    
    query = "SELECT DISTINCT " \
            "PCVH.TradeDate, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_TradarName, " \
            "PCVH.TradeGroupId, " \
            "PCVH.AllInSpread, " \
            "PCVH.DealValue, " \
            "100.0*(PCVH.AllInSpread/PCVH.DealValue) AS spread " \
            "FROM PnLAppDb.dbo.PositionCalculatedValuesHistory AS PCVH " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TARGET ON PCVH.SecurityID= SPT_TARGET.ID " \
            "WHERE SPT_TRDGRP.TargetTickers LIKE ('%'+SPT_TARGET.Ticker+'%') AND PCVH.TradeDate > '" \
            + start_date_yyyy_mm_dd + "' " \
            "AND PCVH.ConsolidatedMarketingGroup = 'Merger Arbitrage' AND PCVH.DealValue <> 0 " \
            "AND PCVH.AllInSpread IS NOT NULL " \
            "ORDER BY TradeDate "
    
    results = optimized_execute(query)
    columns = ["Date", "TradeGroup", "TradeGroup_TradarName", "TradeGroupId", "AllInSpread", "DealValue", "Spread(%)"]
    df = pd.DataFrame(results, columns=columns)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))
    return df

In [57]:
def get_securities_pnl_by_tradegroup_and_fund_current(limit_to_tradegroups=[], limit_to_funds=[],
                                              is_tg_names_in_tradar_format=False, start_date_yyyymmdd=None,
                                              end_date_yyyymmdd=None, rollup_pnl=False):
    slicer = dfutils.df_slicer()
    if (start_date_yyyymmdd is not None) & (type(start_date_yyyymmdd) != str):
            start_date_yyyymmdd = start_date_yyyymmdd.strftime('%Y%m%d')
    elif start_date_yyyymmdd is None:
        start_date_yyyymmdd = '20170101'
    if end_date_yyyymmdd is None:
        now = datetime.datetime.now()
        end_date = slicer.prev_n_business_days(1, now)
        end_date_yyyymmdd = end_date.strftime('%Y%m%d')
    elif type(end_date_yyyymmdd) != str:
        end_date_yyyymmdd = end_date_yyyymmdd.strftime('%Y%m%d')

    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND B.strat IN (" + ",".join([("'" + tg + "'")
                                                           for tg in limit_to_tradegroups 
                                                           if tg is not None]) + ") "
    if is_tg_names_in_tradar_format is True: 
        tg_filter_clause = tg_filter_clause.replace('B.strat', 'SUBSTRING(B.strat,0,21)')
      
    fund_code_filter_clause = ""
    if len([fund for fund in limit_to_funds if fund is not None]) > 0:
        fund_code_filter_clause = " AND F.fund IN (" + ",".join([("'" + fund + "'")
                                                                 for fund in limit_to_funds
                                                                 if fund is not None]) + ") "
        
    # region rollup query
    rollup_sql_query = "DECLARE @Holidays TABLE ([Date] DATE); " \
                        "DECLARE @StartDt INT; " \
                        "DECLARE @EndDt INT; " \
                        "INSERT INTO @Holidays VALUES " \
                        "('2015-01-01'), " \
                        "('2015-01-19'), " \
                        "('2015-02-16'), " \
                        "('2015-04-03'), " \
                        "('2015-05-25'), " \
                        "('2015-07-03'), " \
                        "('2015-09-07'), " \
                        "('2015-10-12'), " \
                        "('2015-11-11'), " \
                        "('2015-11-26'), " \
                        "('2015-12-25'), " \
                        "('2016-01-01'), " \
                        "('2016-01-18'), " \
                        "('2016-02-15'), " \
                        "('2016-03-25'), " \
                        "('2016-05-30'), " \
                        "('2016-07-04'), " \
                        "('2016-09-05'), " \
                        "('2016-11-24'), " \
                        "('2016-12-25'), " \
                        "('2017-01-02'), " \
                        "('2017-01-16'), " \
                        "('2017-02-20'), " \
                        "('2017-04-14'), " \
                        "('2017-05-29'), " \
                        "('2017-07-04'), " \
                        "('2017-09-04'), " \
                        "('2017-10-09'), " \
                        "('2017-11-23'), " \
                        "('2017-12-25'); " \
                        "SET @StartDt = " + start_date_yyyymmdd + "; " \
                        "SET @EndDt = " + end_date_yyyymmdd + "; " \
                        "WITH X AS( " \
                        "SELECT " \
                        "CAST(CONVERT(varchar(8), A.timeKey) AS DATE) as [Date], " \
                        "F.fund, " \
                        "B.strat AS TradeGroup, " \
                        "S.ticker, " \
                        "ST.groupingName as sectype, " \
                        "SUM(pnlFC) as PNL " \
                        "FROM performanceAttributionFact AS A " \
                        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
                        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
                        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
                        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
                        "WHERE A.timeKey >= @StartDt and A.timeKey <= @EndDt " \
                        + tg_filter_clause + \
                        fund_code_filter_clause + \
                        "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.groupingName " \
                        ") " \
                        ", PNL AS ( " \
                        "SELECT " \
                        "X.[Date],  " \
                        "CASE " \
                        "WHEN DATEPART(dw,X.[Date]) IN (7,1) THEN 'WEEKEND' " \
                        "WHEN X.[Date] IN (SELECT * FROM @Holidays) THEN 'HOLIDAY' " \
                        "ELSE 'BUSINESS DAY' END AS [WeekDay], " \
                        "X.Fund, " \
                        "X.TradeGroup, " \
                        "X.Ticker, " \
                        "X.sectype, " \
                        "X.PNL " \
                        "FROM X " \
                        "), " \
                        "Position2EndDate AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "MAX(X.[Date]) AS MaxDate, " \
                        "CASE DATEPART(dw,MAX(X.[Date])) " \
                        "WHEN 6 THEN DATEADD(DAY,3,MAX(X.[Date])) " \
                        "WHEN 7 THEN DATEADD(DAY,2,MAX(X.[Date])) " \
                        "ELSE DATEADD(DAY,1,MAX(X.[Date])) END AS NextBusinessDay " \
                        "FROM PNL AS X " \
                        "GROUP BY X.fund, X.TradeGroup, X.ticker, X.sectype " \
                        "), " \
                        "Position2EndDate_BD AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "X.MaxDate, " \
                        "CASE " \
                        "WHEN NextBusinessDay IN (SELECT * FROM @Holidays) THEN " \
                        "CASE DATEPART(dw,NextBusinessDay) " \
                        "WHEN 6 THEN DATEADD(DAY,3,NextBusinessDay) " \
                        "ELSE DATEADD(DAY,1,NextBusinessDay) END " \
                        "ELSE NextBusinessDay " \
                        "END AS NextBusinessDay_Holiday_Proof " \
                        "FROM Position2EndDate AS X " \
                        "), " \
                        "PNL_DT AS ( " \
                        "SELECT " \
                        "A1.[Date], " \
                        "A1.[WeekDay],  " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.PNL, " \
                        "MAX(A2.[Date]) AS [Last Business Day], " \
                        "DATEDIFF(DAY,MAX(A2.[Date]),A1.[Date]) AS [Last BD days diff] " \
                        "FROM PNL AS A1 " \
                        "LEFT OUTER JOIN PNL AS A2 ON A1.fund = A2.fund and A1.TradeGroup = A2.TradeGroup " \
                        "AND A1.ticker = A2.ticker AND A1.sectype = A2.sectype AND A2.[Date] < A1.[Date] " \
                        "AND A2.[WeekDay] = 'BUSINESS DAY' " \
                        "GROUP BY A1.[Date],A1.[WeekDay],A1.fund, A1.TradeGroup, A1.ticker, A1.sectype, A1.PNL " \
                        "), " \
                        "T AS ( " \
                        "SELECT " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN B.NextBusinessDay_Holiday_Proof ELSE A1.[Date] END AS [Date], " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'BUSINESS DAY' ELSE A1.[WeekDay] END AS [WeekDay], " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.[Last Business Day], " \
                        "A1.PNL, " \
                        "ISNULL(SUM(A2.PNL),0) AS ROLLED_UP_PNL, " \
                        "A1.PNL + ISNULL(SUM(A2.PNL),0) AS TOT_PNL,  " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'Closed on non-BD. Shifted to next BD' ELSE NULL END AS 'Notes' " \
                        "FROM PNL_DT AS A1 " \
                        "LEFT OUTER JOIN PNL_DT AS A2 " \
                        "ON A1.fund = A2.fund AND A1.TradeGroup = A2.TradeGroup AND A1.ticker = A2.ticker " \
                        "AND A1.sectype = A2.sectype AND " \
                        "(A2.[Date] < A1.[Date] AND (A2.[Date] > A1.[Last Business Day] " \
                        "OR A1.[Last Business Day] IS NULL)) " \
                        "AND (A1.[Last BD days diff] <= 4 OR A1.[Last BD days diff] IS NULL) " \
                        "INNER JOIN Position2EndDate_BD AS B ON A1.fund = B.fund AND A1.TradeGroup = B.TradeGroup " \
                        "AND A1.ticker = B.ticker AND A1.sectype = B.sectype " \
                        "WHERE A1.[WeekDay] = 'BUSINESS DAY' OR A1.[Date] = B.MaxDate " \
                        "GROUP BY A1.[Date],A1.[WeekDay], A1.fund,A1.TradeGroup, A1.ticker, " \
                        "A1.sectype, A1.[Last Business Day],A1.[Last BD days diff],A1.PNL, B.MaxDate, " \
                        "B.NextBusinessDay_Holiday_Proof " \
                        ") " \
                        "SELECT " \
                        "T.[Date],  " \
                        "T.fund, " \
                        "T.TradeGroup, " \
                        "T.ticker, " \
                        "T.sectype, " \
                        "T.TOT_PNL " \
                        "FROM T " \
                        "ORDER BY T.fund, T.TradeGroup, T.ticker,T.sectype, T.[Date] "
        
    # endregion
    # region non-rollup query
    # don't take today's pnl - tradar's garbage data
    non_rollup_sql_query = \
        "SELECT " \
        "CAST(CONVERT(VARCHAR(8), A.timeKey) AS DATE) AS [DATE], " \
        "F.fund, " \
        "B.strat AS TradeGroup, " \
        "S.ticker, " \
        "ST.groupingName AS sectype, " \
        "SUM(pnlFC) AS PNL " \
        "FROM performanceAttributionFact AS A " \
        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
        "WHERE A.timeKey >= " + start_date_yyyymmdd + " AND A.timeKey <= " \
        + end_date_yyyymmdd + \
        tg_filter_clause + \
        fund_code_filter_clause + \
        "GROUP BY A.timeKey, B.strat, F.fund,S.ticker,ST.groupingName " \
        "ORDER BY F.fund, B.strat,S.ticker,A.timeKey "
    # endregion
    query = rollup_sql_query if rollup_pnl else non_rollup_sql_query
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'Total P&L']
    try:
        query_result = tradar_optimized_execute(query)
    except Exception as e:
        print(e)
        return pd.DataFrame(columns=cols)

    df = pd.DataFrame(query_result, columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = None
    
    return df

In [47]:
def get_tradegroups_total_pnl(fundcode=None, tg=None, start_date_yyyy_mm_dd=None):
    slicer = dfutils.df_slicer()
    if (start_date_yyyy_mm_dd is not None) & (type(start_date_yyyy_mm_dd) != str):
            start_date_yyyy_mm_dd = start_date_yyyy_mm_dd.strftime('%Y%m%d')
    elif start_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '20150101'
                                                                                        
    query = "SELECT `DATE`, Fund, TradeGroup, pnl, cumpnl FROM " + DB_NAME + ".`tradegroups_pnl_cache`"
    if fundcode is not None and tg is not None:
        query += " WHERE Fund = '" + fundcode + "' AND TradeGroup ='" + tg + "' AND `DATE` >= " + start_date_yyyy_mm_dd
    elif fundcode is not None:
        query += " WHERE Fund = '" + fundcode + "' AND `DATE` >= " + start_date_yyyy_mm_dd
    elif tg is not None:
        query += " WHERE TradeGroup = '" + tg + "' AND `DATE` >= " + start_date_yyyy_mm_dd
    else:
        query += " WHERE `DATE` >= " + start_date_yyyy_mm_dd
        
    res = wic_optimized_execute(query)
    cols = ['Date', 'Fund', 'TradeGroup', 'Total P&L', 'Cumulative Total P&L']
    df = pd.DataFrame(list(res), columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = df['Cumulative Total P&L'].astype(float)
    return df

In [14]:
def mktval_df(pcvh_df):
    float_cols = ['Alpha Exposure', 'AlphaHedge Exposure', 'Hedge Exposure',
                  'Alpha NetMktVal', 'AlphaHedge NetMktVal', 'Hedge NetMktVal',
                  'Alpha GrossMktVal', 'AlphaHedge GrossMktVal', 'Hedge GrossMktVal',
                  'Alpha GrossExp', 'AlphaHedge GrossExp', 'Hedge GrossExp']
    calc_cols = ['Bet Exposure', 'Bet NetMktVal','Bet GrossMktVal', 'Total NetExposure',
                 'Total NetMktVal', 'Total GrossMktVal']

    if len(pcvh_df) == 0: return pd.DataFrame(columns=["Date"]+float_cols+calc_cols)

    alpha_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha']
    alphahedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha Hedge']
    hedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Hedge']

    alpha_net_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Alpha Exposure'})
    alpha_net_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Alpha NetMktVal'})
    alpha_gross_mv = alpha_pcvh[['Date', 'NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Alpha GrossMktVal'})
    alpha_gross_exp = alpha_pcvh[['Date', 'Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Alpha GrossExp'})

    alphahedge_net_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'AlphaHedge Exposure'})
    alphahedge_net_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'AlphaHedge NetMktVal'})
    alphahedge_gross_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'AlphaHedge GrossMktVal'})
    alphahedge_gross_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'AlphaHedge GrossExp'})

    hedge_net_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date','Exposure_Net':'Hedge Exposure'})
    hedge_net_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Hedge NetMktVal'})
    hedge_gross_mv = hedge_pcvh[['Date','NetMktVal']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'NetMktVal':'Hedge GrossMktVal'})
    hedge_gross_exp = hedge_pcvh[['Date','Exposure_Net']].groupby(
        'Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date',
                                                                         'Exposure_Net':'Hedge GrossExp'})

    mktval_df = pd.merge(alpha_net_exp, alphahedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alpha_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, alphahedge_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df = pd.merge(mktval_df, hedge_gross_exp, how='outer', on=['Date']).fillna(0)

    mktval_df[float_cols] = mktval_df[float_cols].astype(float)
    mktval_df['Bet Exposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']
    mktval_df['Bet NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']
    mktval_df['Bet GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']
    mktval_df['Total NetExposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']+mktval_df['Hedge Exposure']
    mktval_df['Total GrossExposure'] = mktval_df['Alpha GrossExp']+mktval_df['AlphaHedge GrossExp']+mktval_df['Hedge GrossExp']
    mktval_df['Total NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']+mktval_df['Hedge NetMktVal']
    mktval_df['Total GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']+mktval_df['Hedge GrossMktVal']


    return mktval_df.sort_values(by='Date')

In [ ]:
def get_tradegroups_snapshot():
    # region query
    query = "SELECT Fund, Sleeve, TradeGroup, Analyst, LongShort, InceptionDate, EndDate, Status," \
            "`Metrics in Bet JSON`,`Metrics in Bet notes JSON`,`Metrics in NAV JSON`,`Metrics in NAV notes JSON` " \
            "FROM " + DB_NAME + ".tradegroups_snapshot2"
    # endregion

    res = wic_optimized_execute(query)
    cols = ['Fund', 'Sleeve', 'TradeGroup', 'Analyst', 'LongShort', 'InceptionDate', 'EndDate', 'Status',
            'Metrics in Bet JSON', 'Metrics in Bet notes JSON', 'Metrics in NAV JSON', 'Metrics in NAV notes JSON']
    return pd.DataFrame(list(res), columns=cols)

def get_current_tradegroup_and_fund():
    query = "SELECT Fund, TradeGroup FROM " + DB_NAME + ".tradegroups_snapshot2"
    
    res = wic_optimized_execute(query)
    cols=['Fund', 'TradeGroup']
    return pd.DataFrame(list(res), columns=cols)

In [ ]:
def df2row(pivot_col, df):
    dfcols = [c for c in df.columns if c != pivot_col]
    cols = [colname + '|' + period for (colname, period) in itertools.product(dfcols, df[pivot_col])]
    cols2vals = {c:None for c in cols}

    for idx in df.index:
        row = df.loc[idx]
        pivot = row[pivot_col]
        for cln in dfcols:
            key = cln + '|' + pivot
            cols2vals[key] = row[cln]
    res = pd.Series()
    for cln in cols:
        res[cln] = cols2vals[cln]
    return res

def json2row(json):
    df = pd.read_json(json)
    return df2row('Period', df)

In [ ]:
def delete_table_by_certain_date(date_yyyy_mm_dd, table_name):
    delete_query = "DELETE from " + DB_NAME + ".`" + table_name + "` WHERE Date = '" + date_yyyy_mm_dd + "' "
    try:
        wic_optimized_execute(delete_query, commit=True)
        return "Success"

    except Exception as e:
        return e

In [20]:
def get_tradegroup_performance_over_own_capital():
    slicer = dfutils.df_slicer()
    today = datetime.datetime.today().strftime('%Y%m%d')
    
    #First, delete the information from the database if yesterday's data exists
    max_date_vol_df = get_max_date("volatility_distribution_timeseries")
    max_date_cap_df = get_max_date("tradegroup_performance_over_own_capital")
    #if max_date_vol_df.strftime('%Y%m%d') == today:
    #    delete_table_by_certain_date(today, 'volatility_distribution_timeseries')
    #if max_date_cap_df.strftime('%Y%m%d') == today:
    #    delete_table_by_certain_date(today, 'volatility_distribution_timeseries')

    try:
        df = get_tradegroups_snapshot()

        df['EndDate'] = df['EndDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%Y-%m-%d'))
        df['InceptionDate'] = df['InceptionDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%Y-%m-%d'))

        # region formatting configuration
        metrics2include = [('P&L(bps)', 'ITD'), ('P&L(bps)', 'YTD'), ('ROMC(bps)', 'YTD'), ('P&L(bps)', 'MTD'),
                                   ('ROMC(bps)', 'MTD'), ('P&L(bps)', '5D'), ('P&L(bps)', '1D'),
                                   ('P&L($)', 'ITD'), ('P&L($)', 'YTD'), ('P&L($)', 'MTD'), ('P&L($)', '5D'), ('P&L($)', '1D'),
                                   ('ANN. VOL', '30D'), ('CAPITAL($)', '1D')]
        metric2display_name = {'P&L(bps)': '', 'P&L($)': '', 'ANN. VOL': 'VOL',
                                       'ROMC(bps)':'ROMC', 'CAPITAL($)':'CAPITAL'}
        metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$', 'ANN. VOL': '%',
                               'ROMC(bps)': 'bps', 'CAPITAL($)':'%'} # will transform capital from $ to % later
        # endregion

        fund2vol_chart = []#, fund2vol_breakdown_by_tg = {}, {}
        funds = ["ARB", "AED", "TACO", "WED", "CAM", "LG", "TAQ", "LEV", "MACO", "MALT"]
        overall_df = pd.DataFrame()
        vol_df = pd.DataFrame()
        nav_df = get_NAV_df_by_date()
        vol_cohort_chart = {}
        for fund_code in funds:
            f_df = df[df['Fund'] == fund_code].copy()
            f_nav = nav_df[nav_df.FundCode == fund_code]
            f_curr_nav = f_nav['NAV'].iloc[-1]

            metrics_df = pd.DataFrame([json2row(j) for j in f_df['Metrics in Bet JSON']])
            metrics_df.index = f_df.index

            for (metric, period) in metrics2include:
                unit = metric2unit[metric]
                disp_name = metric2display_name[metric]
                display_colname = disp_name + ' ' + period + '(' + unit + ')'
                f_df[display_colname] = metrics_df[metric + '|' + period]

            f_df['CAPITAL 1D(%)'] = [np.nan if status == 'CLOSED' else 1e2*(cap_usd/f_curr_nav)
                                        for (cap_usd,status) in zip(f_df['CAPITAL 1D(%)'],f_df['Status'])]

            del f_df['Metrics in NAV JSON']; del f_df['Metrics in NAV notes JSON']
            del f_df['Metrics in Bet JSON']; del f_df['Metrics in Bet notes JSON']
            del f_df['Analyst']

            sleeve2code = {'Merger Arbitrage': 'M&A', 'Equity Special Situations': 'ESS',
                                   'Opportunistic' : 'OPP', 'Forwards':'FWD', 'Credit Opportunities':'CREDIT'}

            f_df['Sleeve'] = f_df['Sleeve'].apply(lambda x: sleeve2code[x] if x in sleeve2code else x)
            f_df = f_df[(~pd.isnull(f_df[' YTD($)']))]  # don't show null ytds
            f_df = f_df.sort_values(by=' YTD($)')
            f_df['Date'] = today

            f_df.rename(columns={' ITD(bps)': 'ITD_bps', ' YTD(bps)': 'YTD_bps', ' MTD(bps)': 'MTD_bps',
                                         ' 5D(bps)': '5D_bps', ' 1D(bps)': '1D_bps', ' ITD($)': 'ITD_Dollar', ' YTD($)': 'YTD_Dollar',
                                         ' MTD($)': 'MTD_Dollar', ' 5D($)': '5D_Dollar', ' 1D($)': '1D_Dollar',
                                         'ROMC YTD(bps)': 'ROMC_YTD_bps', 'ROMC MTD(bps)': 'ROMC_MTD_bps',
                                         'CAPITAL 1D(%)': 'Cap_1D_Pct'}, inplace=True)

            overall_df = overall_df.append(f_df)

                # region VOLATILITY CHARTS
            vol_cohorts = ["0%-5%", "5%-10%", "10%-15%", "15%-20%", "20%-25%", "25%-30%", "30%-35%",
                                   "35%-40%", "40%-45%", "45%-50%", "50%+"]
            vol_cohorts2cnt = {k: 0 for k in vol_cohorts}
            vol_cohorts2tg_vol_pairs = {v: [] for v in vol_cohorts}
            vol_cohort = []
            for (tg, vol) in zip(f_df['TradeGroup'], f_df['VOL 30D(%)']):
                if 0 < vol <= 5:
                    vol_cohorts2cnt["0%-5%"] += 1
                    vol_cohorts2tg_vol_pairs["0%-5%"].append({'bucket': tg, 'value': vol})
                if 5 < vol <= 10:
                    vol_cohorts2cnt["5%-10%"] += 1
                    vol_cohorts2tg_vol_pairs["5%-10%"].append({'bucket': tg, 'value': vol})
                if 10 < vol <= 15:
                    vol_cohorts2cnt["10%-15%"] += 1
                    vol_cohorts2tg_vol_pairs["10%-15%"].append({'bucket': tg, 'value': vol})
                if 15 < vol <= 20:
                    vol_cohorts2cnt["15%-20%"] += 1
                    vol_cohorts2tg_vol_pairs["15%-20%"].append({'bucket': tg, 'value': vol})
                if 20 < vol <= 25:
                    vol_cohorts2cnt["20%-25%"] += 1
                    vol_cohorts2tg_vol_pairs["20%-25%"].append({'bucket': tg, 'value': vol})
                if 25 < vol <= 30:
                    vol_cohorts2cnt["25%-30%"] += 1
                    vol_cohorts2tg_vol_pairs["25%-30%"].append({'bucket': tg, 'value': vol})
                if 30 < vol <= 35:
                    vol_cohorts2cnt["30%-35%"] += 1
                    vol_cohorts2tg_vol_pairs["30%-35%"].append({'bucket': tg, 'value': vol})
                if 35 < vol <= 40:
                    vol_cohorts2cnt["35%-40%"] += 1
                    vol_cohorts2tg_vol_pairs["35%-40%"].append({'bucket': tg, 'value': vol})
                if 40 < vol <= 45:
                    vol_cohorts2cnt["40%-45%"] += 1
                    vol_cohorts2tg_vol_pairs["40%-45%"].append({'bucket': tg, 'value': vol})
                if 45 < vol <= 50:
                    vol_cohorts2cnt["45%-50%"] += 1
                    vol_cohorts2tg_vol_pairs["45%-50%"].append({'bucket': tg, 'value': vol})
                if vol > 50:
                    vol_cohorts2cnt["50%+"] += 1
                    vol_cohorts2tg_vol_pairs["50%+"].append({'bucket': tg, 'value': vol})

            for k in vol_cohorts:
                vol_cohort.append({'bucket': k, 'value': vol_cohorts2cnt[k],
                                        'subdata': vol_cohorts2tg_vol_pairs[k]})

            vol_cohort_chart[fund_code] = vol_cohort
            #endregion
            
        vol_df = pd.DataFrame(columns=['Date','vol_distribution_charts'])
        vol_df.loc[0] = [today, json.dumps(vol_cohort_chart)]
        cols = list(overall_df.columns.values)
        cols.pop(cols.index('Date')) #Want to reorganize so the Date timestamp comes first
        overall_df = overall_df[['Date'] + cols].rename(columns={'VOL 30D(%)': 'VOL_30D_Pct'})
        
    #Import the new additions to the tradegroup_performance_over_own_capital database
    #con = engine.connect()
    #overall_df.to_sql(con=con, schema='wic', name='tradegroup_performance_over_own_capital',
    #                  if_exists='append', chunksize=1000, index=False)
    #con.close()
    #Import the new additions to the volatility_distribution_timeseries database
    #con = engine.connect()
    #vol_df.to_sql(con=con, schema='wic', name=' volatility_distribution_timeseries',
    #              if_exists='append', chunksize=1000, index=False)
    #con.close()

    except Exception as e:
        print e

    return overall_df, vol_df

In [27]:
cap_res, vol_charts = get_tradegroup_performance_over_own_capital()

In [21]:
def get_tradegroups_attribution_to_fund_nav():
    today = datetime.datetime.today().strftime('%Y%m%d')
    #First, delete the information from the database if yesterday's data exists
    max_date_bps = dbutils.wic.get_max_date('tradegroup_attribution_to_fund_nav_bps')
    max_date_dol = dbutils.wic.get_max_date('tradegroup_attribution_to_fund_nav_dollar')
    #if max_date_bps.strftime('%Y%m%d') == today:
    #    dbutils.wic.delete_table_by_certain_date(today, 'tradegroup_attribution_to_fund_nav_bps')
    #if max_date_dol.strftime('%Y%m%d') == today:
    #    dbutils.wic.delete_table_by_certain_date(today, 'tradegroup_attribution_to_fund_nav_dollar')    
    
    df = dbutils.wic.get_tradegroups_snapshot()

    df['EndDate'] = df['EndDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%Y-%m-%d'))
    df['InceptionDate'] = df['InceptionDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%Y-%m-%d'))
    #Do not want tradegroups closed before the start of the current year
    #df_reduced = df[df["EndDate"].dt.year >= today.year] #Takes out deals like cash that don't have date- wrong

    metrics2include = [('P&L(bps)', 'ITD'), ('P&L($)', 'ITD'),
                       ('P&L(bps)', 'YTD'), ('P&L($)', 'YTD'),
                       ('P&L(bps)', 'QTD'), ('P&L($)', 'QTD'),
                       ('P&L(bps)', 'MTD'), ('P&L($)', 'MTD'),
                       ('P&L(bps)', '30D'), ('P&L($)', '30D'),
                       ('P&L(bps)', '5D'), ('P&L($)', '5D'),
                       ('P&L(bps)', '1D'), ('P&L($)', '1D')]

    metric2display_name = {'P&L(bps)': '', 'P&L($)': ''}
    metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$'}

    # unjsonify metrics, and append columns
    metrics_df = pd.DataFrame([dfutils.json2row(json) for json in df['Metrics in NAV JSON']])
    metrics_df.index = df.index


    for (metric, period) in metrics2include:
        unit = metric2unit[metric]
        disp_name = metric2display_name[metric]
        display_colname = disp_name + ' ' + period + '(' + unit + ')'
        df[display_colname] = metrics_df[metric + '|' + period]
        
    del df['Metrics in NAV JSON']; del df['Metrics in NAV notes JSON'];
    del df['Metrics in Bet JSON']; del df['Metrics in Bet notes JSON'];
    del df['Analyst']

    sleeve2code = {'Merger Arbitrage': 'M&A', 'Equity Special Situations': 'ESS',
                   'Opportunistic' : 'OPP', 'Forwards':'FWD', 'Credit Opportunities':'CREDIT'}

    df['Sleeve'] = df['Sleeve'].apply(lambda x: sleeve2code[x] if x in sleeve2code else x)
    df = df[(~pd.isnull(df[' YTD($)']))]  # don't show null ytds. i.e. tradegroups closed before year started
    df['Date'] = today

    base_cols = ['Date', 'Fund', 'Sleeve', 'TradeGroup', 'LongShort', 'InceptionDate', 'EndDate', 'Status']
    bps_cols = [' ITD(bps)', ' YTD(bps)', ' QTD(bps)', ' MTD(bps)', ' 30D(bps)', ' 5D(bps)', ' 1D(bps)']  
    dollar_cols = [' ITD($)', ' YTD($)', ' QTD($)', ' MTD($)', ' 30D($)', ' 5D($)', ' 1D($)']
    bps_df = df[base_cols+bps_cols].sort_values(by=' YTD(bps)')
    bps_df.rename(columns={' ITD(bps)': 'ITD_bps', ' YTD(bps)': 'YTD_bps', ' QTD(bps)': 'QTD_bps',
                           ' MTD(bps)': 'MTD_bps', ' 30D(bps)': '30D_bps', ' 5D(bps)': '5D_bps',
                           ' 1D(bps)': '1D_bps'}, inplace=True)
    dollar_df = df[base_cols+dollar_cols].sort_values(by=' YTD($)')
    dollar_df.rename(columns={' ITD($)': 'ITD_Dollar', ' YTD($)': 'YTD_Dollar', ' QTD($)': 'QTD_Dollar',
                              ' MTD($)': 'MTD_Dollar', ' 30D($)': '30D_Dollar', ' 5D($)': '5D_Dollar',
                              ' 1D($)': '1D_Dollar'}, inplace=True)

    #Import the new additions to the database
    #con = engine.connect()
    #dollar_df.to_sql(con=con, schema='wic', name='tradegroup_attribution_to_fund_nav_dollar',
    #                      if_exists='append', chunksize=1000, index=False)
    #con.close()
    
    #con = engine.connect()
    #bps_df.to_sql(con=con, schema='wic', name='tradegroup_attribution_to_fund_nav_bps',
     #                     if_exists='append', chunksize=1000, index=False)
    #con.close()
    
    return dollar_df

In [74]:
#TradeGroup Attribution to Fund NAV Page
x = get_tradegroups_attribution_to_fund_nav()
fund_and_tradegroup_combos = x[0][["Fund", "TradeGroup"]]
fund_and_tradegroup_combos = fund_and_tradegroup_combos.reset_index()
del fund_and_tradegroup_combos['index']

In [82]:
#unique_tradegroups = x[0]['TradeGroup'].unique()
#limit_to_tradegroups = list(unique_tradegroups)
#unique_funds = x[0]['Fund'].unique()
#limit_to_funds = list(unique_funds)

In [22]:
PCVH = get_position_calculated_values_history()

PCVH retrieval completed....


In [23]:
PCVH

,Date,FundCode,TradeGroup,TradeGroup_Tradar_Name,Sleeve,Bucket,TradeGroupId,Ticker,SecType,MarketCapCategory,...,OutlierNavImpact,Qty,LongShort,Sector,Industry,CatalystRating,CatalystName,Fund_NAV,Equity_Risk_Exp,DealDuration
0,2019-03-01,CAM,PRTK S/S,PRTK S/S,Equity Special Situations,Special Situations,91180.0,PRTK US,EQ,Small Cap,...,0.00000,49710.00,Long,Health Care,Pharmaceuticals,2,Binary,133028338.600000,3.280860e+05,0M-3M
1,2019-03-01,CAM,ZAYO R/R,ZAYO R/R,Equity Special Situations,Re-Ratings,94126.0,ZAYO 190315P00025000,ExchOpt,Mid Cap,...,0.00000,6200.00,Long,Communication Services,Diversified Telecommunication,3,Hard,133028338.600000,-7.329397e+04,0M-3M
2,2019-03-01,CAM,VSM - ENTG,VSM - ENTG,Merger Arbitrage,Conviction,94291.0,VSM US,EQ,Mid Cap,...,0.00000,43531.00,Long,Information Technology,Semiconductors & Semiconductor,1,Soft,133028338.600000,2.074690e+06,0M-3M
3,2019-03-01,LG,RCII M&A BONDS,RCII M&A BONDS,Credit Opportunities,Merger Related Credit,93020.0,RCII 190621P00015000,ExchOpt,Small Cap,...,0.00000,-2900.00,Long,Consumer Discretionary,Specialty Retail,1,Soft,350725850.600000,6.596346e+03,0M-3M
4,2019-03-01,LG,QEP YTC/SPEC M&A BONDS,QEP YTC/SPEC M&A BONDS,Credit Opportunities,Catalyst-Driven Credit,94288.0,QEP US,EQ,Small Cap,...,0.00000,-26381.00,Long,Energy,"Oil, Gas & Consumable Fuels",2,Binary,350725850.600000,-2.092013e+05,0M-3M
5,2019-03-01,LG,KD8 GY S/S,KD8 GY S/S,Equity Special Situations,Special Situations,87453.0,KD8 GR,EQ,Mid Cap,...,0.00000,10591.00,Long,Communication Services,Media,2,Binary,350725850.600000,1.263906e+06,Yr+
6,2019-03-01,LG,WH R/R,WH R/R,Equity Special Situations,Re-Ratings,92933.0,HLT US,EQ,Large Cap,...,0.00000,-4848.00,Long,Consumer Discretionary,"Hotels, Restaurants & Leisure",1,Soft,350725850.600000,-4.061654e+05,0M-3M
7,2019-03-01,LG,ARNC R/R,ARNC R/R,Equity Special Situations,Re-Ratings,94410.0,ARNC US,EQ,Mid Cap,...,0.00000,109992.00,Long,Industrials,Aerospace & Defense,3,Hard,350725850.600000,2.056850e+06,0M-3M
8,2019-03-01,LG,VSM - ENTG,VSM - ENTG,Merger Arbitrage,Conviction,94291.0,VSM 190621P00040000,ExchOpt,Mid Cap,...,0.00000,20400.00,Long,Information Technology,Semiconductors & Semiconductor,1,Soft,350725850.600000,-2.537534e+05,0M-3M
9,2019-03-01,MALT,IDTI - RENESAS,IDTI - RENESAS,Merger Arbitrage,Optimized,93507.0,IDTI US,EQ,Mid Cap,...,-0.68000,5756.00,Long,Information Technology,Semiconductors & Semiconductor,3,Hard,12662150.540000,0.000000e+00,0M-3M


In [29]:
SECURITIES_PNL_DF = get_securities_pnl_by_tradegroup_and_fund_current(limit_to_tradegroups=['MFGP R/R SHORT'],
                                                            is_tg_names_in_tradar_format=True, rollup_pnl=True)

In [33]:
SECURITIES_PNL_DF[SECURITIES_PNL_DF.Ticker == 'PRGS US'].sort_values(by='Date')

,Date,Fund,TradeGroup,Ticker,SecType,Total P&L,Cumulative Total P&L
1707,2018-12-26,TAQ,MFGP R/R Short,PRGS US,Equity,222.6231,None
1708,2018-12-27,TAQ,MFGP R/R Short,PRGS US,Equity,170.1000,None
1709,2018-12-28,TAQ,MFGP R/R Short,PRGS US,Equity,24.5700,None
1071,2018-12-31,LG,MFGP R/R Short,PRGS US,Equity,598.5880,None
1710,2018-12-31,TAQ,MFGP R/R Short,PRGS US,Equity,79.3800,None
219,2018-12-31,AED,MFGP R/R Short,PRGS US,Equity,226.7060,None
647,2018-12-31,CAM,MFGP R/R Short,PRGS US,Equity,213.8190,None
1711,2019-01-01,TAQ,MFGP R/R Short,PRGS US,Equity,0.0000,None
648,2019-01-01,CAM,MFGP R/R Short,PRGS US,Equity,0.0000,None
1072,2019-01-01,LG,MFGP R/R Short,PRGS US,Equity,0.0000,None


In [ ]:
def get_exposure_spreads_nav_df(PCVH):
    start = time.time()
    slicer = dfutils.df_slicer()
    today = datetime.datetime.today()
    yesterday = slicer.prev_n_business_days(1, today).strftime('%Y%m%d')
    
    #First, delete the information from the database if yesterday's data exists
    max_date = get_max_date("tradegroup_exposure_nav_info")
    #if max_date.strftime('%Y%m%d') == yesterday:
      #  delete_table_by_certain_date(yesterday, 'tradegroup_exposure_nav_info')

    fund_tg_combos = get_active_tradegroups_from_tradar()
    
    last_date_exposure_db = datetime.datetime.strptime(max_date.strftime('%Y%m%d'), '%Y%m%d')
    FUND_NAV_DF = get_NAV_df_by_date(start_date_yyyy_mm_dd=last_date_exposure_db)
    SPREADS_HISTORY_DF = get_spread_history_by_date(start_date_yyyy_mm_dd=last_date_exposure_db)
    PCVH = PCVH[PCVH.Date > last_date_exposure_db]
    PCVH['TradeGroup'] = PCVH['TradeGroup'].apply(lambda x: x.upper())
    SPREADS_HISTORY_DF['TradeGroup'] = SPREADS_HISTORY_DF['TradeGroup'].apply(lambda x: x.upper())
    overall_df = pd.DataFrame()
    
    for i in range(0, len(fund_tg_combos['TradeGroup'])):
        overall_exp_df = pd.DataFrame()
        tradar_tg_name = fund_tg_combos['TradeGroup'][i]
        fund = fund_tg_combos['Fund'][i]

        tg_pcvh = PCVH[(PCVH.TradeGroup == tradar_tg_name) & (PCVH.FundCode == fund)].copy()
        fund_nav_df = FUND_NAV_DF[FUND_NAV_DF.FundCode == fund].copy()
        spreads_history_df = SPREADS_HISTORY_DF[SPREADS_HISTORY_DF.TradeGroup_TradarName == tradar_tg_name].copy()
        
        # Capital AS (%) OF NAV TIME SERIES #
        cap_df = pd.DataFrame()
        capital_df = tg_pcvh[tg_pcvh['AlphaHedge']
                             .isin(['Alpha',
                                    'Alpha Hedge'])][['Date',
                                                      'NetMktVal']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index()
        if len(capital_df) > 0:
            capital_df = pd.merge(capital_df, fund_nav_df, how='inner', on=['Date'])
            capital_df['CapitalAsPctOfNAV'] = 100.0*(capital_df['NetMktVal'].astype(float)
                                                 /capital_df['NAV'].astype(float))
    
            cap_df['Date'] = capital_df['Date']
            cap_df['Capital_Percent_of_NAV'] = capital_df['CapitalAsPctOfNAV']
            cap_df['Fund'] = fund
            cap_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(cap_df, sort=True)
            else:
                overall_exp_df = overall_exp_df.merge(cap_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        # NETEXP (% OF NAV) #
        net_exp_df = pd.DataFrame()
        tot_net_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
        tot_net_exp_df = tot_net_exp_df.rename(columns={'Exposure_Net':'Total Exposure'}).sort_values(by='Date')
        tot_net_exp_df = pd.merge(tot_net_exp_df, fund_nav_df, how='inner', on=['Date'])
        tot_net_exp_df['NetExpAsPctOfNAV'] = 100.0*(tot_net_exp_df['Total Exposure'].astype(float)
                                                    /tot_net_exp_df['NAV'].astype(float))
        if len(tot_net_exp_df) > 0:
            net_exp_df['Date'] = tot_net_exp_df['Date']
            net_exp_df['NetExp_Percent_of_NAV'] = tot_net_exp_df['NetExpAsPctOfNAV']
            net_exp_df['Fund'] = fund
            net_exp_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(net_exp_df, sort=True)
            else:
                overall_exp_df = overall_exp_df.merge(net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])

        # GROSSEXP (% OF NAV) #
        gross_exp_df = pd.DataFrame()
        tot_gross_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index()
        tot_gross_exp_df = tot_gross_exp_df.rename(columns={'Exposure_Net':
                                                            'Total Gross Exposure'}).sort_values(by='Date')
        tot_gross_exp_df = pd.merge(tot_gross_exp_df, fund_nav_df, how='inner', on=['Date'])
        tot_gross_exp_df['GrossExpAsPctOfNAV'] = 100.0*(tot_gross_exp_df['Total Gross Exposure'].astype(float)
                                                        /tot_gross_exp_df['NAV'].astype(float))
        if len(tot_gross_exp_df) > 0:
            gross_exp_df['Date'] = tot_gross_exp_df['Date']
            gross_exp_df['GrossExp_Percent_of_NAV'] = tot_gross_exp_df['GrossExpAsPctOfNAV']
            gross_exp_df['Fund'] = fund
            gross_exp_df['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(gross_exp_df, sort=True)
            else:
                overall_exp_df = overall_exp_df.merge(gross_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
           
        hedge_exp_df = pd.DataFrame()
        if len(tg_pcvh) > 0:
            alpha_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                    'Alpha'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
            alphahedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                         'Alpha Hedge'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
            hedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] ==
                                    'Hedge'][['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()

            if len(alpha_net_exp) > 0:
                alpha_net_exp_df = pd.DataFrame()
                alpha_net_exp = pd.merge(alpha_net_exp, fund_nav_df, how='inner', on=['Date'])
                alpha_net_exp['Exposure_Net'] = 100.0*(alpha_net_exp['Exposure_Net'].astype(float)
                                                       /alpha_net_exp['NAV'].astype(float))
                alpha_net_exp_df['Date'] = alpha_net_exp['Date']
                alpha_net_exp_df['Alpha_Exposure'] = alpha_net_exp['Exposure_Net']
                alpha_net_exp_df['Fund'] = fund
                alpha_net_exp_df['TradeGroup'] = tradar_tg_name
                
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(alpha_net_exp_df, sort=True)
                else:
                    overall_exp_df = overall_exp_df.merge(alpha_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
            
            if len(alphahedge_net_exp) > 0:
                alphahedge_net_exp_df = pd.DataFrame()
                alphahedge_net_exp = pd.merge(alphahedge_net_exp, fund_nav_df, how='inner', on=['Date'])
                alphahedge_net_exp['Exposure_Net'] = 100.0*(alphahedge_net_exp['Exposure_Net'].astype(float)
                                                            /alphahedge_net_exp['NAV'].astype(float))
                alphahedge_net_exp_df['Date'] = alphahedge_net_exp['Date']
                alphahedge_net_exp_df['AlphaHedge_Exposure'] = alphahedge_net_exp['Exposure_Net']
                alphahedge_net_exp_df['Fund'] = fund
                alphahedge_net_exp_df['TradeGroup'] = tradar_tg_name
            
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(alphahedge_net_exp_df, sort=True)
                else:
                    overall_exp_df = overall_exp_df.merge(alphahedge_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])

            if len(hedge_net_exp) > 0:
                hedge_net_exp_df = pd.DataFrame()
                hedge_net_exp = pd.merge(hedge_net_exp, fund_nav_df, how='inner', on=['Date'])
                hedge_net_exp['Exposure_Net'] = 100.0*(hedge_net_exp['Exposure_Net'].astype(float)
                                                       /hedge_net_exp['NAV'].astype(float))
                hedge_net_exp_df['Date'] = hedge_net_exp['Date']
                hedge_net_exp_df['Hedge_Exposure'] = hedge_net_exp['Exposure_Net']
                hedge_net_exp_df['Fund'] = fund
                hedge_net_exp_df['TradeGroup'] = tradar_tg_name
                
                if overall_exp_df.empty:
                    overall_exp_df = overall_exp_df.append(hedge_net_exp_df, sprt=True)
                else:
                    overall_exp_df = overall_exp_df.merge(hedge_net_exp_df, how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        #SPREAD AS PERCENT#
        spread = pd.DataFrame()
        if tradar_tg_name in spreads_history_df['TradeGroup_TradarName'].unique():
            spreads_ts_df = spreads_history_df[spreads_history_df['TradeGroup_TradarName'] ==
                                               tradar_tg_name][['Date', 'Spread(%)']].sort_values(by='Date')
            spreads_ts_df = spreads_ts_df.rename(columns={'Spread(%)':'SpreadAsPct'}).sort_values(by='Date')
            spreads_ts_df['SpreadAsPct'] = spreads_ts_df['SpreadAsPct'].astype(float)
            
            spread['Date'] = spreads_ts_df['Date']
            spread['Spread_as_Percent'] = spreads_ts_df['SpreadAsPct']
            spread['Fund'] = fund
            spread['TradeGroup'] = tradar_tg_name
            
            if overall_exp_df.empty:
                overall_exp_df = overall_exp_df.append(spread, sort=True)
            else:
                overall_exp_df = overall_exp_df.merge(spread, how='outer', on=['Date', 'Fund', 'TradeGroup'])
                
        overall_df = overall_df.append(overall_exp_df, sort=True)  
    
    #Import the new additions to the database
    #con = engine.connect()
    #overall_df.to_sql(con=con, schema='wic', name='tradegroup_exposure_nav_info', if_exists='append',
    #                  chunksize=1000, index=False)
    #con.close()
    
    end = time.time()
    print(end - start)
    return overall_df

In [ ]:
new_additions = get_exposure_spreads_nav_df(PCVH)
new_additions

In [48]:
def daily_pnl_df(PCVH):
    slicer = dfutils.df_slicer()
    today = datetime.datetime.today()
    yesterday = slicer.prev_n_business_days(1, today).strftime('%Y%m%d')
    
    #First, delete the information from the database if yesterday's data exists
    max_date_pnl = get_max_date('tradegroup_overall_pnl')
    if max_date_pnl.strftime('%Y%m%d') == yesterday:
        dbutils.wic.delete_table_by_certain_date(yesterday, 'ticker_pnl_breakdown')
        dbutils.wic.delete_table_by_certain_date(yesterday, 'tradegroup_overall_pnl')    
    

    fund_tg_combos = get_active_tradegroups_from_tradar()
    
    limit_to_tradegroups = list(fund_tg_combos['TradeGroup'].unique())
    limit_to_funds = list(fund_tg_combos['Fund'].unique())
    
    #Get the start date for the queries (30 days back because this takes into account rolling volatility)
    start_date = datetime.datetime.strptime(max_date_pnl.strftime('%Y%m%d'), '%Y%m%d')
    start_date_yyyy_mm_dd = slicer.prev_n_business_days(30, start_date)
    
    PCVH = PCVH[PCVH.Date > start_date_yyyy_mm_dd]
    FUND_NAV_DF = get_NAV_df_by_date(start_date_yyyy_mm_dd=start_date_yyyy_mm_dd)
    TG_PNL_DF = get_tradegroups_total_pnl(start_date_yyyy_mm_dd=start_date_yyyy_mm_dd)
    SECURITIES_PNL_DF = get_securities_pnl_by_tradegroup_and_fund_current(limit_to_tradegroups=limit_to_tradegroups,
                                                                          limit_to_funds=limit_to_funds,
                                                                          is_tg_names_in_tradar_format=True,
                                                                          start_date_yyyymmdd=start_date_yyyy_mm_dd,
                                                                          rollup_pnl=True)
    SECURITIES_PNL_DF['TradeGroup'] = SECURITIES_PNL_DF['TradeGroup'].apply(lambda x: x.upper())
    TG_PNL_DF['TradeGroup'] = TG_PNL_DF['TradeGroup'].apply(lambda x: x.upper())
    PCVH['TradeGroup'] = PCVH['TradeGroup'].apply(lambda x: x.upper())
    fund_tg_combos['TradeGroup'] = fund_tg_combos['TradeGroup'].apply(lambda x: x.upper())
    
    final_pnl_df = pd.DataFrame()
    final_ticker_pnl_df = pd.DataFrame()
    for i in range(0, len(fund_tg_combos['TradeGroup'])):
        bps_dollar_pnl_df = pd.DataFrame()
        bps_dollar_ticker_pnl_df = pd.DataFrame()
        tradar_tg_name = fund_tg_combos['TradeGroup'][i]
        fund = fund_tg_combos['Fund'][i]
        
        tg_pcvh = PCVH[(PCVH.TradeGroup == tradar_tg_name) & (PCVH.FundCode == fund)].copy()
        fund_nav_df = FUND_NAV_DF[FUND_NAV_DF.FundCode == fund].copy()
        securities_pnl_df = SECURITIES_PNL_DF[(SECURITIES_PNL_DF.TradeGroup == tradar_tg_name)
                                              & (SECURITIES_PNL_DF.Fund == fund)].copy()
        tg_pnl_df = TG_PNL_DF[(TG_PNL_DF.TradeGroup == tradar_tg_name) & (TG_PNL_DF.Fund == fund)].copy()
        
        options_pnl_df = securities_pnl_df[securities_pnl_df['SecType'] ==
                                           'Option'][['Date', 'Total P&L']].groupby('Date').sum().reset_index().copy()
        options_pnl_df = options_pnl_df.iloc[np.trim_zeros(options_pnl_df['Total P&L']).index].copy()

        if len(options_pnl_df) >= 1:
            options_pnl_df = options_pnl_df.rename(columns={'Total P&L': 'Options_PnL_Dollar'})
            options_pnl_df['Fund'] = fund
            options_pnl_df['TradeGroup'] = tradar_tg_name
            options_pnl_df = pd.merge(options_pnl_df, fund_nav_df, how='inner', on=['Date'])
            options_pnl_df['Shifted Capital'] = options_pnl_df['NAV'].shift(1)
            if pd.isnull(options_pnl_df['Shifted Capital'].iloc[0]):
                options_pnl_df.loc[0, 'Shifted Capital'] = options_pnl_df['NAV'].iloc[0]
            options_pnl_df['Shifted Capital'] = options_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            options_pnl_df['Shifted Forward-Filled Capital'] = options_pnl_df['Shifted Capital'].ffill()
            options_pnl_df['Options_PnL_bps'] = 1e4*(options_pnl_df["Options_PnL_Dollar"]/
                                                     options_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                                -np.inf],
                                                                                                               np.nan)
            if bps_dollar_pnl_df.empty:
                bps_dollar_pnl_df = bps_dollar_pnl_df.append(options_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Options_PnL_Dollar', 'Options_PnL_bps']], sort=True)
            else:
                bps_dollar_pnl_df = pd.merge(bps_dollar_pnl_df, options_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Options_PnL_Dollar', 'Options_PnL_bps']],
                                             how='outer', on=['Date', 'Fund', 'TradeGroup'])

        #P&L Breakdown by Ticker -- to be kept in a separate db
        tg_securities = securities_pnl_df[securities_pnl_df['SecType'] != 'Option']['Ticker'].unique()
        for ticker in tg_securities:
            if ticker == 'USD':
                continue
            tkr_pnl_df = securities_pnl_df[securities_pnl_df['Ticker'] == ticker].copy()
            #tkr_pnl_df = tkr_pnl_df.loc[np.trim_zeros(tkr_pnl_df['Total P&L']).index].copy()
            if len(tkr_pnl_df) < 1:
                continue
            tkr_pnl_df = tkr_pnl_df.rename(columns={'Total P&L': 'Ticker_PnL_Dollar'})
            tkr_pnl_df['Fund'] = fund
            tkr_pnl_df['TradeGroup'] = tradar_tg_name
            tkr_pnl_df = pd.merge(tkr_pnl_df, fund_nav_df, how='inner', on=['Date'])
            tkr_pnl_df['Shifted Capital'] = tkr_pnl_df['NAV'].shift(1)
            if pd.isnull(tkr_pnl_df['Shifted Capital'].iloc[0]):
                tkr_pnl_df.loc[0, 'Shifted Capital'] = tkr_pnl_df['NAV'].iloc[0]
            tkr_pnl_df['Shifted Capital'] = tkr_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            tkr_pnl_df['Shifted Forward-Filled Capital'] = tkr_pnl_df['Shifted Capital'].ffill()
            tkr_pnl_df['Ticker_PnL_bps'] = 1e4*(tkr_pnl_df["Ticker_PnL_Dollar"]/
                                                     tkr_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                            -np.inf],
                                                                                                            np.nan)  
            bps_dollar_ticker_pnl_df = bps_dollar_ticker_pnl_df.append(tkr_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Ticker_PnL_Dollar', 'Ticker_PnL_bps', 'Ticker']])

        final_ticker_pnl_df = final_ticker_pnl_df.append(bps_dollar_ticker_pnl_df, sort=True)

        #Daily PnL  
        daily_pnl_df = tg_pnl_df[['Date', 'Fund', 'TradeGroup', 'Total P&L']]
        if len(daily_pnl_df) >= 1:
            daily_pnl_df = daily_pnl_df.rename(columns={'Total P&L': 'Daily_PnL_Dollar'})
            daily_pnl_df['Fund'] = fund
            daily_pnl_df['TradeGroup'] = tradar_tg_name
            daily_pnl_df = pd.merge(daily_pnl_df, fund_nav_df, how='inner', on=['Date'])
            daily_pnl_df['Shifted Capital'] = daily_pnl_df['NAV'].shift(1)
            if pd.isnull(daily_pnl_df['Shifted Capital'].iloc[0]):
                daily_pnl_df.loc[0, 'Shifted Capital'] = daily_pnl_df['NAV'].iloc[0]
            daily_pnl_df['Shifted Capital'] = daily_pnl_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
            daily_pnl_df['Shifted Forward-Filled Capital'] = daily_pnl_df['Shifted Capital'].ffill()
            daily_pnl_df['Daily_PnL_bps'] = 1e4*(daily_pnl_df["Daily_PnL_Dollar"]/
                                                     daily_pnl_df["Shifted Forward-Filled Capital"]).replace([np.inf, -np.inf],
                                                                                                             np.nan)  
            if bps_dollar_pnl_df.empty:
                bps_dollar_pnl_df = bps_dollar_pnl_df.append(daily_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Daily_PnL_Dollar', 'Daily_PnL_bps']], sort=True)
            else:
                bps_dollar_pnl_df = pd.merge(bps_dollar_pnl_df, daily_pnl_df[['Date', 'Fund', 'TradeGroup',
                                                                         'Daily_PnL_Dollar', 'Daily_PnL_bps']], how='outer',
                                             on=['Date', 'Fund', 'TradeGroup'])

        rolling_vol_df = pd.DataFrame()
        if len(tg_pcvh) > 0:
                tg_capital_df = mktval_df(tg_pcvh)
                roc_df = pd.merge(tg_pnl_df, tg_capital_df[['Date', 'Bet GrossMktVal']], how='inner', on=['Date'])
                if len(roc_df) >= 1:
                    roc_df['Shifted Capital'] = roc_df['Bet GrossMktVal'].shift(1)
                    if pd.isnull(roc_df['Shifted Capital'].iloc[0]):
                        roc_df.loc[0, 'Shifted Capital'] = roc_df['Bet GrossMktVal'].iloc[0]
                    roc_df['Shifted Capital'] = roc_df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
                    roc_df['Shifted Forward-Filled Capital'] = roc_df['Shifted Capital'].ffill()
                    roc_df['PnL_Over_Cap_bps'] = 1e4*(roc_df["Total P&L"]/
                                               roc_df["Shifted Forward-Filled Capital"]).replace([np.inf, -np.inf],
                                                                                                 np.nan) 
                    roc_df['Rolling_30D_PnL_Vol'] = np.sqrt(252)*(roc_df['PnL_Over_Cap_bps']/100.0).rolling(window=30).std()
                    if len(roc_df['Rolling_30D_PnL_Vol'].dropna()) > 0:
                        rolling_vol_df[['Date', 'Fund', 'TradeGroup', 'Rolling_30D_PnL_Vol']] = roc_df[
                            ["Date", "Fund", "TradeGroup", "Rolling_30D_PnL_Vol"]].dropna()

                        if bps_dollar_pnl_df.empty:
                            bps_dollar_pnl_df = bps_dollar_pnl_df.append(rolling_vol_df[['Date', 'Fund', 'TradeGroup',
                                                                                         'Rolling_30D_PnL_Vol']], sort=True)
                        else:
                            bps_dollar_pnl_df = bps_dollar_pnl_df.merge(rolling_vol_df[['Date', 'Fund',
                                                                                        'TradeGroup',
                                                                                        'Rolling_30D_PnL_Vol']],
                                                                        how='outer', on=['Date', 'Fund', 'TradeGroup'])

                    if bps_dollar_pnl_df.empty:
                        bps_dollar_pnl_df = bps_dollar_pnl_df.append(roc_df[['Date', 'Fund', 'TradeGroup',
                                                                         'PnL_Over_Cap_bps']], sort=True)
                    else:
                        bps_dollar_pnl_df = bps_dollar_pnl_df.merge(roc_df[['Date', 'Fund', 'TradeGroup', 'PnL_Over_Cap_bps']],
                                                                    how='outer', on=['Date', 'Fund', 'TradeGroup'])
        
        final_pnl_df = final_pnl_df.append(bps_dollar_pnl_df, sort=True)
        
    overall_pnl_additions = final_pnl_df[final_pnl_df.Date > start_date]
    ticker_pnl_additions = final_ticker_pnl_df[final_ticker_pnl_df.Date > start_date]
    
    con = engine.connect()
    ticker_pnl_additions.to_sql(con=con, schema='wic', name='ticker_pnl_breakdown',
                         if_exists='append', chunksize=1000, index=False)
    con.close()
    
    con = engine.connect()
    overall_pnl_additions.to_sql(con=con, schema='wic', name='tradegroup_overall_pnl',
                         if_exists='append', chunksize=1000, index=False)
    con.close()

    return overall_pnl_additions, ticker_pnl_additions

In [49]:
pnl_result_current, pnl_ticker_results_current = daily_pnl_df(PCVH)

2019-04-02 00:00:00


C:\Users\aduprey\AppData\Local\Continuum\anaconda3\envs\Python3Env\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [64]:
pnl_result_current

,Daily_PnL_Dollar,Daily_PnL_bps,Date,Fund,Options_PnL_Dollar,Options_PnL_bps,PnL_Over_Cap_bps,Rolling_30D_PnL_Vol,TradeGroup
27,-627.640000,-0.049610,2019-05-15,AED,NaN,NaN,-13.018224,NaN,AABA R/R SHORT
27,-619.740000,-0.047825,2019-05-15,CAM,NaN,NaN,-13.061848,NaN,AABA R/R SHORT
27,-1796.720000,-0.051922,2019-05-15,LG,NaN,NaN,-13.032538,NaN,AABA R/R SHORT
27,-67.220000,-0.292308,2019-05-15,TAQ,NaN,NaN,-12.876610,NaN,AABA R/R SHORT
27,-97.740000,-0.148964,2019-05-15,WED,NaN,NaN,-13.211004,NaN,AABA R/R SHORT
10,458.750000,1.994888,2019-05-15,TAQ,-152.5,-0.663151,90.176865,NaN,ACOR R/R
21,2625.402600,0.207517,2019-05-15,AED,NaN,NaN,8.958692,NaN,ADSW - WM
21,52407.030000,0.298931,2019-05-15,ARB,NaN,NaN,9.339975,NaN,ADSW - WM
21,2581.404900,0.199207,2019-05-15,CAM,NaN,NaN,8.958392,NaN,ADSW - WM
19,374.590000,0.665644,2019-05-15,LEV,NaN,NaN,9.382304,NaN,ADSW - WM


In [65]:
#con = engine.connect()
#pnl_result_current.to_sql(con=con, schema='wic', name='tradegroup_overall_pnl',
#                         if_exists='append', chunksize=1000, index=False)
#con.close()

In [66]:
pnl_ticker_results_current

,Date,Fund,Ticker,Ticker_PnL_Dollar,Ticker_PnL_bps,TradeGroup
27,2019-05-15,AED,AABA US,-7273.720000,-0.574928,AABA R/R SHORT
27,2019-05-15,AED,BABA US,6646.080000,0.525319,AABA R/R SHORT
27,2019-05-15,CAM,AABA US,-7158.180000,-0.552397,AABA R/R SHORT
27,2019-05-15,CAM,BABA US,6538.440000,0.504572,AABA R/R SHORT
27,2019-05-15,LG,AABA US,-20799.320000,-0.601064,AABA R/R SHORT
27,2019-05-15,LG,BABA US,19002.600000,0.549142,AABA R/R SHORT
27,2019-05-15,TAQ,AABA US,-787.580000,-3.424816,AABA R/R SHORT
27,2019-05-15,TAQ,BABA US,720.360000,3.132507,AABA R/R SHORT
27,2019-05-15,WED,AABA US,-1116.180000,-1.701156,AABA R/R SHORT
27,2019-05-15,WED,BABA US,1018.440000,1.552192,AABA R/R SHORT


In [67]:
#con = engine.connect()
#pnl_ticker_results_current.to_sql(con=con, schema='wic', name='ticker_pnl_breakdown',
#                         if_exists='append', chunksize=1000, index=False)
#con.close()